### 기본적으로 사용할 라이브러리를 불러온다.

In [1]:
#import library
import numpy as np
import pandas as pd

### 학습데이터를 불러온다.

In [2]:

train_data = pd.read_csv("train.csv")

# 학습용데이터에 대한 전처리

### column중에 범주형 자료를 담은 column에서 범주를 줄이기 위해 국가를 대륙으로 매핑하는과정에서 필요한 딕셔너리

In [3]:
# column중에 범주형 자료를 담은 column에서 범주를 줄이기 위해 국가를 대륙으로 매핑하는과정에서 필요한 딕셔너리
country_continent_dict = {
 'US': 'US',
 'Cuba': 'North America',
 'Portugal': 'Europe',
 'Mexico': 'North America',
 'Unknown': 'US',
 'Puerto-Rico': 'North America',
 'Germany': 'Europe',
 'Japan': 'Asia',
 'Poland': 'Europe',
 'Columbia': 'South America',
 'Philippines': 'Asia',
 'Italy': 'Europe',
 'Trinadad&Tobago': 'South America',
 'England': 'Europe',
 'South Korea': 'Asia',
 'Iran': 'Asia',
 'France': 'Europe',
 'India': 'Asia',
 'China': 'Asia',
 'Dominican-Republic': 'North America',
 'Scotland': 'Europe',
 'Ecuador': 'South America',
 'Nicaragua': 'North America',
 'Peru': 'South America',
 'Cambodia': 'Asia',
 'Canada': 'North America',
 'Jamaica': 'North America',
 'Vietnam': 'Asia',
 'Hong Kong': 'Asia',
 'Thailand': 'Asia',
 'Haiti': 'North America',
 'Guatemala': 'North America',
 'Laos': 'Asia',
 'Yugoslavia': 'Europe',
 'Ireland': 'Europe',
 'El-Salvador': 'North America',
 'Panama': 'North America',
 'Honduras': 'North America',
 'Greece': 'Europe',
 'Outlying-U S (Guam USVI etc)': 'US',
 'Hungary': 'Europe',
 'Taiwan': 'Asia',
 'Holand-Netherlands': 'Europe'
}

### 위의 메핑을 적용시킬 열: 'Birth_Country', 'Birth_Country (Mother)', 'Birth_Country (Father)'

In [ ]:

columns_to_update = ['Birth_Country', 'Birth_Country (Mother)', 'Birth_Country (Father)']

### 각 열에 대해 국가를 대륙으로 매핑


In [ ]:
for column in columns_to_update:
    train_data[column] = train_data[column].map(country_continent_dict)

### 독립변수 열들에서 데이터 전처리

In [ ]:
# Age 열의 데이터가 17이하 75이상은 제거
train_data = train_data[train_data['Age'].between(17, 75)]

#'Employment Status'열의 데이터가 'Not Working' 혹은 'Seeking Full-Time'이면 제거
train_data = train_data[~train_data['Employment_Status'].isin(['Not Working', 'Seeking Full-Time'])]

# 'Gains', 'Losses', 'Dividends', 'Household_Status', 'Income_Status' 열들을 제거
train_data.drop(['Gains', 'Losses', 'Dividends', 'Household_Status', 'Income_Status'], axis=1, inplace=True)

# Map 'Gender' values from 'M' and 'F' to 0 and 1, respectively
train_data['Gender'] = train_data['Gender'].map({'M': 0, 'F': 1})

#ID열은 income에 전혀 영향을 주지 못하므로 drop메소드로 제거한다.
edit_train_data = edit_train_data.drop(columns=['ID'])


### education열에 대한 전처리

In [ ]:
# 'Education_Status'열에서 범주의 수가 너무 많아 몇개의 범주로 합친다.
education_map = {
    'High graduate': 'High', 'High Senior': 'High', 
    'High Junior': 'High', 'High Sophomore': 'High',
    'Elementary (5-6)': 'Elementary(1-6)', 'Elementary (1-4)': 'Elementary(1-6)',
    'Kindergarten': 'Baby', 'Children': 'Baby'
}
train_data['Education_Status'] = train_data['Education_Status'].replace(education_map)
#원 핫 인코딩을 진행할 범주형 자료를 담은 열들의 리스트
columns=[
    'Education_Status',
    'Employment_Status',
    'Industry_Status',
    'Occupation_Status',
    'Race',
    'Hispanic_Origin',
    'Martial_Status',
    'Household_Summary',
    'Citizenship',
    'Birth_Country',
    'Birth_Country (Father)',
    'Birth_Country (Mother)',
    'Tax_Status'
]
#학습데이터를 get_dummies메소드를 이용하여 원 핫 인코딩 한다.
edit_train_data=pd.get_dummies(train_data,columns=columns,dtype=int)

### 학습데이터에서 종속변수열에 대한 전처리

In [ ]:
# 학습시킬 데이터에서 종속변수 income이 이상치(2500이상)를 가지고 있다면 제거
train_data = train_data[train_data['Income'] <= 2500]
#학습시킬 데이터에 종속변수가 들어있으면 않되므로 drop시킨다.
edit_train_data = edit_train_data.drop(columns=['Income'])

### 완성된 학습용 데이터를 파일로 저장

In [ ]:
edit_train_data.to_csv("editedTrain.csv", index=False)

# lightGBM모델을 이용하기 위해 최적의 하이퍼파라미터를 찾는 과정

### 모델 이용을 위한 기본 세팅

In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

# 데이터 불러오기 edit_train_data는 'Income'열을 drop했으므로 train_data에서 'Income'열을 가져온다.
X = edit_train_data
y = train_data['Income']

# 위의 불러온 train.csv의 전처리를 거친 데이터 셋에서 학습용과 테스트용으로 데이터를 분리한다.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Fitting 4 folds for each of 100 candidates, totalling 400 fits



KeyboardInterrupt



### 랜덤서치를 통한 하이처파라미터 튜닝

##### 랜덤 서치는 하이퍼파라미터 튜닝을 위한 방법 중 하나로, 무작위로 선택된 하이퍼파라미터 조합을 사용하여 모델을 평가한다.
##### 그리드 서치는 리드 서치는 사전에 정의된 그리드에서 모든 하이퍼파라미터 조합을 시도하므로 cost가 크다.

In [ ]:
# LightGBM 모델 생성
lgbm_model = LGBMRegressor()
#랜덤 서치를 위한 파라미터 분포 설정
param_dist = {
    'learning_rate': np.logspace(-3, 0, 100),  # 로그 스케일로 학습률 분포 설정
    'n_estimators': [100, 200, 300, 400, 500],  # 트리의 수
    'max_depth': [3, 5, 7, 9],  # 트리의 최대 깊이
    'num_leaves': [15, 31, 63, 127],  # 각 트리의 최대 잎의 수
    'min_child_samples': [10, 20, 30, 50],  # 각 리프 노드에 필요한 최소 데이터 수
    'subsample': [0.6, 0.7, 0.8, 0.9, 1.0],  # 샘플링 비율
    'colsample_bytree': [0.6, 0.7, 0.8, 0.9, 1.0]  # 각 트리를 구성하는 특성의 비율
}

# 랜덤 서치 수행
random_search = RandomizedSearchCV(
    estimator=lgbm_model,
    param_distributions=param_dist,
    n_iter=100,  # 시도할 하이퍼파라미터 조합의 수
    cv=5,  # 교차 검증 폴드 수
    scoring='neg_mean_squared_error',  # 평가 지표
    n_jobs=-1,  # 모든 CPU 코어 사용
    verbose=2,  # 실행 과정 표시
    random_state=42  # 랜덤 시드 고정
)

# 랜덤 서치 수행
random_search.fit(X_train, y_train)

# 최적의 모델 및 파라미터 출력
print("Best Parameters:", random_search.best_params_)

### 테스트 데이터에 대한 예측 수행 및 성능 평가

In [ ]:

best_model = random_search.best_estimator_
y_pred = best_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)
# 성능 평가를 위해 rmse와 R²값을 출력한다.
print("RMSE:", rmse)
print("R²:", r2)

# test data set에 실제 적용

### test.csv를 불러온다

In [ ]:
testData = pd.read_csv("test.csv")

### 전처리 과정

In [ ]:
# 국가를 대륙으로 매핑하는 딕셔너리
country_continent_dict = {
 'US': 'US',
 'Cuba': 'North America',
 'Portugal': 'Europe',
 'Mexico': 'North America',
 'Unknown': 'US',
 'Puerto-Rico': 'North America',
 'Germany': 'Europe',
 'Japan': 'Asia',
 'Poland': 'Europe',
 'Columbia': 'South America',
 'Philippines': 'Asia',
 'Italy': 'Europe',
 'Trinadad&Tobago': 'South America',
 'England': 'Europe',
 'South Korea': 'Asia',
 'Iran': 'Asia',
 'France': 'Europe',
 'India': 'Asia',
 'China': 'Asia',
 'Dominican-Republic': 'North America',
 'Scotland': 'Europe',
 'Ecuador': 'South America',
 'Nicaragua': 'North America',
 'Peru': 'South America',
 'Cambodia': 'Asia',
 'Canada': 'North America',
 'Jamaica': 'North America',
 'Vietnam': 'Asia',
 'Hong Kong': 'Asia',
 'Thailand': 'Asia',
 'Haiti': 'North America',
 'Guatemala': 'North America',
 'Laos': 'Asia',
 'Yugoslavia': 'Europe',
 'Ireland': 'Europe',
 'El-Salvador': 'North America',
 'Panama': 'North America',
 'Honduras': 'North America',
 'Greece': 'Europe',
 'Outlying-U S (Guam USVI etc)': 'US',
 'Hungary': 'Europe',
 'Taiwan': 'Asia',
 'Holand-Netherlands': 'Europe'
}

# 업데이트할 열 목록: 본인 출신국가, 엄마 출신국가, 아빠 출신국가
columns_to_update = ['Birth_Country', 'Birth_Country (Mother)', 'Birth_Country (Father)']

# 각 열에 대해 국가를 대륙으로 매핑
for column in columns_to_update:
    testData[column] = testData[column].map(country_continent_dict)


# Drop the 'Gains', 'Losses', 'Dividends', 'Household_Status', 'Income_Status' columns
testData.drop(['Gains', 'Losses', 'Dividends', 'Household_Status', 'Income_Status'], axis=1, inplace=True)

# Map 'Gender' values from 'M' and 'F' to 0 and 1, respectively
testData['Gender'] = testData['Gender'].map({'M': 0, 'F': 1})

# Consolidate education levels and rename as specified
education_map = {
    'High graduate': 'High', 'High Senior': 'High', 
    'High Junior': 'High', 'High Sophomore': 'High',
    'Elementary (5-6)': 'Elementary(1-6)', 'Elementary (1-4)': 'Elementary(1-6)',
    'Kindergarten': 'Baby', 'Children': 'Baby'
}
testData['Education_Status'] = testData['Education_Status'].replace(education_map)
columns=[
    'Education_Status',
    'Employment_Status',
    'Industry_Status',
    'Occupation_Status',
    'Race',
    'Hispanic_Origin',
    'Martial_Status',
    'Household_Summary',
    'Citizenship',
    'Birth_Country',
    'Birth_Country (Father)',
    'Birth_Country (Mother)',
    'Tax_Status'
]
#제거코드(ID)
testData = testData.drop(columns=['ID'])

### test에서는 행을 제거할 수 없으므로 예측이 끝나고 income이 0이 되기로 판정한 행번호 목록을 작성한다.

In [ ]:
row_to_zero = testData[(testData["Age"] < 17) | 
                      (testData["Age"] > 75) | 
                      (testData["Employment_Status"] == "Not Working") | 
                      (testData["Employment_Status"] == "Seeking Full-Time")].index

### 범주형 데이터를 담은 열들에 대해 train에는 있고 test에는 없는 범주가 있어 원핫 인코딩 시 열의 갯수가 맞지 않는 에러가 발생하므로 에러처리 

In [6]:
# 'df_encoded'는 훈련 데이터에 대해 이미 원핫 인코딩이 수행된 DataFrame이다.
# 'train_cols'는 원핫 인코딩된 훈련 데이터의 열 순서이다.
train_cols = edit_train_data.columns

# 'columns' 리스트에 지정된 열에 대해 testData 데이터프레임을 원핫 인코딩한다.
edited_testData = pd.get_dummies(testData, columns=columns, dtype=int)

# 훈련 데이터에만 있는 열을 찾는다.
missing_cols = set(train_cols) - set(edited_testData.columns)

# 훈련 데이터에만 있는 열을 테스트 데이터에 추가하고, 해당 열의 값을 0으로 설정한다.
for col in missing_cols:
    edited_testData[col] = 0

# 테스트 데이터에서 훈련 데이터에 없는 열을 찾는니다.
extra_cols = set(edited_testData.columns) - set(train_cols)

# 테스트 데이터에서 훈련 데이터에 없는 열을 제거한다.
edited_testData = edited_testData.drop(columns=extra_cols)

# 테스트 데이터의 열 순서를 훈련 데이터의 열 순서와 동일하게 재정렬한다.
edited_testData = edited_testData[train_cols]

# 이제 'edited_testData'는 원핫 인코딩된 테스트 데이터이며, 열 순서가 'df_encoded'와 일치한다.


# 최종 모델을 이용한 예측

### 결과에서 이전에 얻어둔 row_to_zero와 예측값이 음수인 행들의 값을 전부 0으로 처리해준다.

In [7]:
y_pred_lightGBM = best_model.predict(edited_testData) 
for i in range(len(y_pred_lightGBM)):
    if i in row_to_zero:
        y_pred_lightGBM[i]=0
y_pred_lightGBM[y_pred_lightGBM < 0] = 0

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).


# 제출파일 작성

In [8]:
# ID 생성
ids = [f'TEST_{i:04d}' for i in range(len(y_pred_lightGBM))]

# 데이터프레임 생성
df = pd.DataFrame({
    'ID': ids,
    'Income': y_pred_lightGBM
})

# CSV 파일로 저장
df.to_csv('submissionFilelightGBM.csv', index=False)